In [2]:
#Install required packages
!pip install langchain langchain-google-genai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.0 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

1. Query Rewriting: Reformulating queries to improve retrieval.

In [4]:
re_write_llm = ChatGoogleGenerativeAI (temperature=0, model="gemini-2.0-flash", max_tokens=4000)

# Create a prompt template for query rewriting
query_rewrite_template = """You are an AI assistant tasked with reformulating user queries to improve retrieval in a RAG system.
Given the original query, rewrite it to be more specific, detailed, and likely to retrieve relevant information.

Original query: {original_query}

Rewritten query:"""

query_rewrite_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=query_rewrite_template
)

# Create an LLMChain for query rewriting
query_rewriter = query_rewrite_prompt | re_write_llm

def rewrite_query(original_query):
    """
    Rewrite the original query to improve retrieval.

    Args:
    original_query (str): The original user query

    Returns:
    str: The rewritten query
    """
    response = query_rewriter.invoke(original_query)
    return response.content

implement on a use case

In [5]:
# example query over the understanding climate change dataset
original_query = "What are the impacts of climate change on the environment?"
rewritten_query = rewrite_query(original_query)
print("Original query:", original_query)
print("\nRewritten query:", rewritten_query)

Original query: What are the impacts of climate change on the environment?

Rewritten query: What are the specific environmental impacts of anthropogenic climate change, including but not limited to effects on biodiversity, sea levels, ocean acidification, extreme weather events (such as hurricanes, droughts, and floods), and changes in ecosystem distribution and function? Please provide examples and cite relevant scientific studies or reports.


2. Step-back Prompting: Generating broader queries for better context retrieval.

In [6]:
step_back_llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-2.0-flash", max_tokens=4000)


# Create a prompt template for step-back prompting
step_back_template = """You are an AI assistant tasked with generating broader, more general queries to improve context retrieval in a RAG system.
Given the original query, generate a step-back query that is more general and can help retrieve relevant background information.

Original query: {original_query}

Step-back query:"""

step_back_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=step_back_template
)

# Create an LLMChain for step-back prompting
step_back_chain = step_back_prompt | step_back_llm

def generate_step_back_query(original_query):
    """
    Generate a step-back query to retrieve broader context.

    Args:
    original_query (str): The original user query

    Returns:
    str: The step-back query
    """
    response = step_back_chain.invoke(original_query)
    return response.content

implement on a use case

In [7]:
# example query over the understanding climate change dataset
original_query = "What are the impacts of climate change on the environment?"
step_back_query = generate_step_back_query(original_query)
print("Original query:", original_query)
print("\nStep-back query:", step_back_query)

Original query: What are the impacts of climate change on the environment?

Step-back query: What is climate change?


3. Sub-query Decomposition: Breaking complex queries into simpler sub-queries.

In [8]:
sub_query_llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-2.0-flash", max_tokens=4000)

# Create a prompt template for sub-query decomposition
subquery_decomposition_template = """You are an AI assistant tasked with breaking down complex queries into simpler sub-queries for a RAG system.
Given the original query, decompose it into 2-4 simpler sub-queries that, when answered together, would provide a comprehensive response to the original query.

Original query: {original_query}

example: What are the impacts of climate change on the environment?

Sub-queries:
1. What are the impacts of climate change on biodiversity?
2. How does climate change affect the oceans?
3. What are the effects of climate change on agriculture?
4. What are the impacts of climate change on human health?"""


subquery_decomposition_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=subquery_decomposition_template
)

# Create an LLMChain for sub-query decomposition
subquery_decomposer_chain = subquery_decomposition_prompt | sub_query_llm

def decompose_query(original_query: str):
    """
    Decompose the original query into simpler sub-queries.

    Args:
    original_query (str): The original complex query

    Returns:
    List[str]: A list of simpler sub-queries
    """
    response = subquery_decomposer_chain.invoke(original_query).content
    sub_queries = [q.strip() for q in response.split('\n') if q.strip() and not q.strip().startswith('Sub-queries:')]
    return sub_queries

implement on a use case

In [9]:
# example query over the understanding climate change dataset
original_query = "What are the impacts of climate change on the environment?"
sub_queries = decompose_query(original_query)
print("\nSub-queries:")
for i, sub_query in enumerate(sub_queries, 1):
    print(sub_query)


Sub-queries:
Okay, I understand. Here's a breakdown of the query "What are the impacts of climate change on the environment?" into simpler sub-queries:
**Sub-queries:**
1.  How does climate change affect global temperatures and weather patterns?
2.  What are the effects of climate change on water resources (e.g., glaciers, rivers, rainfall)?
3.  How does climate change impact ecosystems and natural habitats?
4.  What are the effects of climate change on air quality and atmospheric composition?



Sentiment analysis can be a valuable addition to a RAG system's query transformation pipeline by providing insights into the user's emotional state or attitude towards the query topic. This information can be leveraged to improve the relevance and tone of the retrieved information.

**1. Benefits of Analyzing Query Sentiment:**

Analyzing the sentiment of a user's query offers several benefits:

*   **Improved Relevance:** Understanding the sentiment can help the system prioritize documents or passages that align with the user's emotional context. A negative query might benefit from retrieval of troubleshooting guides or explanations of issues, while a positive query might benefit from user testimonials or success stories.
*   **Tailored Responses:** The RAG system can tailor the retrieved information and the final generated response to match the user's sentiment. For instance, a query with negative sentiment might warrant a more empathetic and problem-solution oriented response.
*   **Enhanced User Experience:** By acknowledging and responding to the user's underlying sentiment, the RAG system can provide a more personalized and helpful interaction, leading to a better user experience.
*   **Identifying Urgency or Severity:** Extreme negative sentiment might indicate a critical issue or an urgent need for specific information, allowing the system to prioritize retrieval of crucial documents.

**2. Specific Uses of Sentiment Information in Retrieval:**

Sentiment information can influence the retrieval process in several ways:

*   **Filtering or Ranking Search Results:** Retrieved documents can be filtered or ranked based on their own sentiment score relative to the query's sentiment. For a negative query about a product, the system might prioritize retrieving reviews or forum discussions that also express negative sentiment, as these might contain relevant problem descriptions or warnings. Conversely, for a positive query, it might prioritize positive reviews or marketing materials.
*   **Selecting Knowledge Bases or Document Subsets:** Depending on the sentiment, the system could be directed to search within specific subsets of the knowledge base. A negative query about a product might trigger a search primarily within support forums or bug reports, while a positive query might focus on product features or success stories.
*   **Adjusting Retrieval Parameters:** Sentiment could influence retrieval parameters like the desired level of detail or the type of information sought. A query with high frustration might lead the system to look for concise, direct answers and troubleshooting steps.
*   **Influencing Reranking:** After initial retrieval, a reranking step could heavily weigh documents that match the query's sentiment, bringing the most emotionally relevant information to the top.

**Example:** If a user queries "This product is terrible, why does it keep crashing?", the highly negative sentiment could trigger the retrieval system to:

*   Prioritize documents from support forums or troubleshooting guides related to product crashes.
*   Filter out overly positive marketing materials.
*   Look for specific error codes or common issues reported by other users.
*   Rank user reviews that also mention crashing issues higher.

**3. Potential Challenges and Limitations:**

Using sentiment analysis in RAG systems also presents challenges:

*   **Accuracy of Sentiment Analysis:** Sentiment analysis can be complex, especially with nuanced language, sarcasm, or domain-specific jargon. Inaccurate sentiment detection can lead to suboptimal retrieval.
*   **Context Dependency:** The sentiment of a query can be highly context-dependent. A negative word might not indicate negative sentiment in all situations (e.g., "critically acclaimed").
*   **Ambiguous Queries:** Some queries may have neutral or mixed sentiment, making it difficult to use sentiment as a strong retrieval signal.
*   **Data Requirements:** Training or fine-tuning sentiment models for specific domains or query types might require labeled data.
*   **Integration Complexity:** Integrating sentiment analysis seamlessly into the existing RAG pipeline requires careful design and implementation.
*   **Potential for Bias:** The sentiment analysis model itself might have biases that could affect retrieval fairness or accuracy.

## Implement sentiment analysis





Import the necessary library for sentiment analysis and define a function to perform sentiment analysis on a given query. NLTK is a suitable choice for this task.



In [12]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download VADER lexicon for sentiment analysis
nltk.download('vader_lexicon')

def analyze_sentiment(query: str):
    """
    Analyzes the sentiment of a given query.

    Args:
        query (str): The user query.

    Returns:
        str: A sentiment category (positive, negative, neutral).
    """
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(query)

    # Determine the sentiment category based on compound score
    if sentiment_scores['compound'] >= 0.05:
        return 'positive'
    elif sentiment_scores['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...



Test the `analyze_sentiment` function with a few example queries to ensure it works as expected.



In [13]:
# Test the sentiment analysis function
query1 = "I love this product, it's amazing!"
query2 = "This is the worst service I have ever received."
query3 = "The weather is neutral today."
query4 = "I am not happy with the results."

sentiment1 = analyze_sentiment(query1)
sentiment2 = analyze_sentiment(query2)
sentiment3 = analyze_sentiment(query3)
sentiment4 = analyze_sentiment(query4)

print(f"Query: '{query1}'\nSentiment: {sentiment1}\n")
print(f"Query: '{query2}'\nSentiment: {sentiment2}\n")
print(f"Query: '{query3}'\nSentiment: {sentiment3}\n")
print(f"Query: '{query4}'\nSentiment: {sentiment4}\n")

Query: 'I love this product, it's amazing!'
Sentiment: positive

Query: 'This is the worst service I have ever received.'
Sentiment: negative

Query: 'The weather is neutral today.'
Sentiment: neutral

Query: 'I am not happy with the results.'
Sentiment: negative



## Define query routing for rag


lets Determine how query routing will be used to direct queries to the most appropriate retrieval method or data source.


# Enhancing RAG Retrieval with Query Routing

Query routing is a critical component in advanced RAG systems that allows the system to intelligently direct incoming user queries to the most appropriate retrieval method or data source. Instead of searching across a monolithic knowledge base with a single strategy, routing enables a more nuanced and efficient approach to information retrieval.

**1. Benefits of Query Routing:**

Implementing query routing offers several significant advantages:

*   **Improved Relevance:** By directing queries to specialized knowledge bases or retrieval methods best suited for the query's nature, the system can retrieve more accurate and relevant information. For example, a query about a specific product feature might be routed to a product documentation knowledge base, while a query about general industry trends might be routed to a collection of news articles.
*   **Increased Efficiency:** Routing can significantly reduce the search space, leading to faster retrieval times and lower computational costs. Instead of searching a vast, heterogeneous dataset, the system focuses on a smaller, more relevant subset.
*   **Enhanced Scalability:** As the RAG system grows and incorporates more diverse data sources and retrieval techniques, routing provides a structured way to manage this complexity and ensure efficient operation.
*   **Flexibility and Adaptability:** Query routing allows for easy integration of new data sources or retrieval methods without requiring a complete overhaul of the system. New routes can be added to handle specific types of queries or access new information.
*   **Optimized Resource Utilization:** Different retrieval methods have varying computational requirements. Routing can direct queries to the most resource-efficient method that is still likely to provide a good answer.

**2. Specific Criteria for Routing:**

Query routing can be based on various criteria extracted from the user query:

*   **Query Type:** Classifying queries based on their intent (e.g., factual question, comparison, instructional query, troubleshooting). A factual question might be routed to a structured knowledge base, while an instructional query might be directed to a collection of tutorials or guides.
*   **Domain or Topic:** Identifying the subject matter of the query (e.g., finance, healthcare, technology, specific product). Queries related to a specific domain can be routed to specialized knowledge bases for that domain.
*   **User Intent:** Understanding the underlying goal of the user (e.g., seeking information, solving a problem, making a decision). This can help route the query to resources that align with that intent.
*   **Sentiment:** As discussed previously, the sentiment of the query can also be a routing criterion. Negative sentiment might route to troubleshooting guides, while positive sentiment might route to testimonials or success stories.
*   **Keywords or Entities:** The presence of specific keywords, named entities (like product names or company names), or technical terms can be used to route queries to relevant data sources.
*   **Query Complexity:** Simple queries might be handled by a basic retrieval method, while complex queries requiring multi-hop reasoning or synthesis might be routed to more sophisticated techniques or multiple data sources.

**3. Different Routing Strategies:**

Various strategies can be employed for query routing:

*   **Rule-Based Routing:** This involves defining a set of explicit rules based on the query criteria. For example, a rule might state: "If the query contains 'product X' and 'troubleshooting', route to the product X support documentation." This is straightforward to implement but can become complex to manage with a large number of rules.
*   **Learned Routing (e.g., using Machine Learning Classifiers):** A machine learning model (like a classifier) can be trained to predict the best route based on features extracted from the query. This can be more flexible and scalable than rule-based systems, especially for handling nuanced queries. The model learns the optimal routing strategy from data.
*   **Hybrid Approaches:** Combining rule-based and learned strategies can leverage the strengths of both. Simple, clear-cut cases can be handled by rules, while more ambiguous queries can be routed by a learned model.
*   **LLM-Based Routing:** A large language model (LLM) can be used to analyze the query and determine the most appropriate route. The LLM can understand the query's context and intent more deeply than simpler methods.

**4. Potential Challenges and Considerations:**

Implementing effective query routing requires careful consideration of several challenges:

*   **Defining Routing Criteria:** Identifying the most effective criteria for routing and how to accurately extract them from queries can be challenging.
*   **Maintaining and Updating Routing Logic:** As the knowledge base and retrieval methods evolve, the routing logic needs to be updated accordingly.
*   **Handling Ambiguous Queries:** Queries that don't clearly fit into a specific category can be difficult to route effectively.
*   **Performance Overhead:** The routing process itself adds some overhead to the query processing pipeline. It's important to ensure that the routing logic is efficient.
*   **Evaluating Routing Effectiveness:** Determining how well the routing strategy is performing requires metrics to assess whether queries are being sent to the most appropriate destinations.
*   **Data Source Interoperability:** Ensuring that the retrieved information from different data sources can be seamlessly integrated and synthesized for the final response.

By carefully designing and implementing a query routing mechanism, a RAG system can significantly improve its performance, efficiency, and ability to provide relevant and timely information to users.


## Implement query routing


 implement the query routing logic based on the query's characteristics or sentiment.


**Reasoning**:
Define the `route_query` function with conditional logic based on sentiment and test it with example queries.



In [15]:
def route_query(query: str, sentiment: str):
    """
    Routes a query based on its sentiment.

    Args:
        query (str): The original user query.
        sentiment (str): The sentiment of the query ('positive', 'negative', 'neutral').

    Returns:
        str: The determined route for the query.
    """
    print(f"Original query: '{query}'")
    print(f"Query sentiment: {sentiment}")

    if sentiment == 'positive':
        route = "Route: Positive sentiment - Directing to success stories and testimonials."
    elif sentiment == 'negative':
        route = "Route: Negative sentiment - Directing to troubleshooting guides and support resources."
    else:  # Assuming 'neutral' or any other sentiment
        route = "Route: Neutral sentiment - Directing to general information."

    print(route)
    return route

# Test cases
route_query("I love this product, it's amazing!", "positive")
route_query("This is the worst service I have ever received.", "negative")
route_query("The weather is neutral today.", "neutral")
route_query("I am not happy with the results.", "negative") # Testing a negative sentiment

Original query: 'I love this product, it's amazing!'
Query sentiment: positive
Route: Positive sentiment - Directing to success stories and testimonials.
Original query: 'This is the worst service I have ever received.'
Query sentiment: negative
Route: Negative sentiment - Directing to troubleshooting guides and support resources.
Original query: 'The weather is neutral today.'
Query sentiment: neutral
Route: Neutral sentiment - Directing to general information.
Original query: 'I am not happy with the results.'
Query sentiment: negative
Route: Negative sentiment - Directing to troubleshooting guides and support resources.


'Route: Negative sentiment - Directing to troubleshooting guides and support resources.'

## Integrate techniques


 how to integrate all five techniques (Query Rewriting, Step-back Prompting, Sub-query Decomposition, Sentiment Analysis, and Query Routing) into a cohesive RAG workflow.


# Integrated RAG Workflow with Query Transformation Techniques

This section outlines a possible workflow for a Retrieval Augmented Generation (RAG) system that integrates the five query transformation techniques: Query Rewriting, Step-back Prompting, Sub-query Decomposition, Sentiment Analysis, and Query Routing. The goal is to leverage these techniques to improve the relevance and effectiveness of the information retrieval and response generation process.

**1. Workflow Overview**

The proposed workflow involves a sequence of steps where the original user query is analyzed and transformed in multiple ways before engaging with the retrieval and generation components of the RAG system. The order of the steps is designed to build a richer understanding of the user's intent and information needs.

Here is a possible order and explanation of the workflow:

*   **Step 1: Receive Original Query:** The process begins when the RAG system receives the user's initial query.

*   **Step 2: Sentiment Analysis:** The first transformation applied is Sentiment Analysis.
    *   **Purpose:** To understand the emotional tone of the user's query. This provides early context about the user's state and can inform subsequent routing and response generation.
    *   **Output:** The sentiment of the query (e.g., 'positive', 'negative', 'neutral').

*   **Step 3: Query Routing:** Based on the sentiment and potentially other initial query characteristics (like keywords or a preliminary query type classification), the system performs Query Routing.
    *   **Purpose:** To direct the query processing down the most appropriate path or towards the most relevant initial data sources. This allows for specialized handling of queries based on sentiment or domain.
    *   **Output:** A determined 'route' or strategy for subsequent steps (e.g., prioritizing certain knowledge bases for negative sentiment, focusing on product features for positive sentiment). This step might influence which of the other transformation techniques are emphasized or how their outputs are utilized.

*   **Step 4: Query Rewriting:** The original query is then passed through the Query Rewriting process.
    *   **Purpose:** To create a more specific, detailed, and optimized version of the query for direct retrieval. This rewritten query aims to improve the precision of the initial search.
    *   **Output:** A reformulated version of the original query.

*   **Step 5: Step-back Prompting:** Simultaneously or in parallel with Query Rewriting (depending on implementation), Step-back Prompting is applied to the original query.
    *   **Purpose:** To generate a more general or fundamental query related to the original. This helps retrieve broader context and background information that might be necessary for a comprehensive answer, especially for complex topics.
    *   **Output:** A broader, more general query.

*   **Step 6: Sub-query Decomposition:** For queries identified as potentially complex (either through initial classification or analysis during previous steps), Sub-query Decomposition is performed.
    *   **Purpose:** To break down a multifaceted query into several simpler, more manageable sub-queries. This allows the system to retrieve specific pieces of information related to different aspects of the original complex query.
    *   **Output:** A list of simpler sub-queries.

*   **Step 7: Parallel Retrieval:** The system now utilizes the outputs from the transformation steps to perform retrieval. This is where the routing decision from Step 3 becomes crucial.
    *   **Input for Retrieval:** The rewritten query (from Step 4), the step-back query (from Step 5), and the list of sub-queries (from Step 6, if applicable). Based on the route determined in Step 3, the system might prioritize one or more of these transformed queries and direct the search to specific knowledge bases or document subsets. For instance, a negative sentiment route might prioritize retrieval using the rewritten query on troubleshooting documents, while also using the step-back query to provide basic product information.
    *   **Purpose:** To retrieve relevant documents or passages from the knowledge base(s) using the transformed queries.
    *   **Output:** A collection of retrieved documents or passages.

*   **Step 8: Information Synthesis and Reranking:** The retrieved documents are then processed. This might involve synthesizing information from multiple documents and reranking them based on relevance, redundancy, and potentially the original query's sentiment (e.g., highlighting information that addresses negative sentiment).
    *   **Purpose:** To consolidate information from various sources and prioritize the most relevant content for the final response.
    *   **Output:** A refined set of retrieved information.

*   **Step 9: Response Generation:** The refined retrieved information, along with the original user query and potentially the determined sentiment and route, are passed to the language model for final response generation.
    *   **Purpose:** To generate a coherent, informative, and contextually appropriate response to the user's original query, leveraging the retrieved information and potentially tailoring the tone based on the query's sentiment.
    *   **Output:** The final generated response to the user.

**2. Logical Ordering and Integration**

The proposed order is logical because it progressively refines the understanding of the user's need:

*   **Sentiment Analysis First:** Understanding sentiment early allows the system to set a potential emotional context for the interaction and inform subsequent routing.
*   **Routing Based on Initial Analysis:** Routing immediately after sentiment (and potentially a quick query type analysis) allows the system to choose the most relevant set of tools and data sources for the query, optimizing the rest of the pipeline.
*   **Parallel Query Transformations:** Applying Query Rewriting, Step-back Prompting, and Sub-query Decomposition in parallel (or in close sequence) generates multiple perspectives on the original query, increasing the chances of retrieving comprehensive and relevant information. The routing decision might influence *which* of these transformations are prioritized or how their results are weighted.
*   **Retrieval Leveraging Multiple Inputs:** Using the outputs of the transformations for retrieval, guided by the routing strategy, ensures that the search is both specific (rewriting), broad (step-back), and detailed (sub-queries) as needed.
*   **Synthesis and Generation:** Finally, synthesizing the retrieved information and generating the response brings together all the insights gained from the initial query analysis and the retrieval process.

**3. How Outputs are Used**

*   **Sentiment:** Informs routing, can influence reranking of retrieved documents, and helps tailor the tone of the final generated response.
*   **Query Routing:** Determines which knowledge bases are searched, which transformation techniques are emphasized, and how the retrieved information is weighted or filtered.
*   **Rewritten Query:** Used for a precise, targeted search for highly relevant documents.
*   **Step-back Query:** Used for a broader search to retrieve foundational or contextual information.
*   **Sub-queries:** Each sub-query is used for targeted retrieval of information related to specific aspects of a complex original query. The results from these individual retrievals are then combined.

**4. Query Routing's Influence**

Query routing, potentially influenced by sentiment analysis, significantly impacts the overall retrieval strategy:

*   It acts as a traffic controller, directing queries to specialized pipelines.
*   A negative sentiment query might be routed to a 'support' pipeline that prioritizes troubleshooting guides and uses the rewritten query with a focus on problem-solving keywords.
*   A positive sentiment query might be routed to a 'marketing/features' pipeline that prioritizes product descriptions and testimonials, potentially emphasizing retrieval using the rewritten query and downplaying the need for step-back information.
*   A complex, neutral query might be routed to a pipeline that heavily utilizes sub-query decomposition across multiple general knowledge bases.

By integrating these techniques and strategically routing the query, the RAG system can move beyond simple keyword matching to a more intelligent, context-aware, and user-sentiment-informed retrieval and generation process.

## Summary:

### Data Analysis Key Findings

*   Sentiment analysis can be a valuable addition to a RAG system by providing insights into the user's emotional state, which can help improve relevance, tailor responses, and enhance user experience. Specific uses include filtering/ranking results, selecting knowledge bases, adjusting parameters, and influencing reranking based on query sentiment.
*   Query routing allows the RAG system to intelligently direct queries to the most appropriate retrieval method or data source based on criteria such as query type, domain, intent, sentiment, keywords, and complexity. This leads to improved relevance, increased efficiency, enhanced scalability, and greater flexibility.
*   An integrated RAG workflow can incorporate Sentiment Analysis and Query Routing early in the process. Sentiment analysis helps understand the user's emotional tone, which then informs query routing. Query routing directs the subsequent steps, potentially influencing how other techniques like Query Rewriting, Step-back Prompting, and Sub-query Decomposition are applied and which data sources are prioritized for retrieval.

### Insights or Next Steps

*   Implement the integrated RAG workflow by developing components for sentiment analysis and query routing and connecting them with the existing query transformation techniques and retrieval/generation modules.
*   Evaluate the effectiveness of the integrated system by comparing its performance (relevance, response quality, efficiency) against a baseline RAG system without these techniques, using relevant metrics and user feedback.
